<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-1020-dev/PPK_Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Step 0.  Required first step. Load functions and defs. 
if __name__ == '__main__':
  PPK_Utilities_asof = 'PPK_Utilities_asof:2020-1020-1547'
  print(PPK_Utilities_asof)

import pathlib
import importlib
import subprocess
import os

#@title def teqc_install()  {form-width: "35%"} 
# Get and install teqc
def teqc_install():
  import os
  import subprocess as sp
  state = 'initial.'
  os.chdir('/content/')
  print(f'teqc_install(): ', end='')
  #rv = sp.run(f'which teqc', shell=True)
  rv = !which teqc
  if rv == []:
    print('Downloading and installing Teqc from Unavco.  ', end='')
    sp.run(f'wget https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip', shell=True)
    sp.run(f'unzip teqc_Lx86_64s.zip', shell=True)
    sp.run(f'mv teqc /usr/local/bin', shell=True)
    sp.run(f'rm -rf teqc_Lx86_64s.zip', shell=True)
    state = ' Teqc installed.'
  else:
    a = 1;
    state='Teqc was already installed.'
  print(state)

  try:
    import google.colab
    IN_COLAB = True
  except:
    IN_COLAB = False

############################  Tree
if __name__ == '__main__':
  rv = ! which tree
  if rv == []:
    ! apt-get install tree
  else:
    print('    tree was already installed.')
############################  Tree

############################ RTKlib
if __name__ == '__main__':
  rv = ! which rnx2rtkp
  if rv == []:
    !cd /usr/local/bin; rm  -rf convbin rnx2rtkp pos2kml rtkrcv str2str
    !cd /content/;       rm -rf RTKLIB
    !cd /usr/local/src/; rm -rf RTKLIB
    !cd /usr/local/src; git clone https://github.com/lidar532/RTKLIB.git
    !cd /usr/local/src/RTKLIB/app/; make install
    !cd /usr/local/bin; chmod uog+x rnx2rtkp convbin pos2kml rtkrcv str2str
    rtk_lib_loaded = True
  else:
    rtk_lib_loaded = True
    print('  RTKlib was already installed.')
############################ RTKlib

############################ Georinex
  # Install geprinex package if not already loaded.
  if importlib.util.find_spec('georinex') is None:
    !pip install georinex
  else:
    print('georinex was already installed.')
############################ Georinex

if __name__ == '__main__':
  teqc_install()


In [ ]:
#@title Step 1. Mount your Google Drive as /content/Gdrive 
from google.colab import drive
!rm -rf /content/sample_data/
drive.mount('/content/.gdrive')
!ln -sf /content/.gdrive/My\ Drive/ /content/Gdrive
print('Your Gdrive is mounted')

# PPK Utilities 

# Project Tools

In [ ]:
#@title P1.0 **(Optional)** Setup A new Project Directory Tree {form-width: "25%"}
if __name__ == '__main__':
  My_Project = '2020-0927-MARKS' #@param {type:"string"}

  rv = !rm -rf /content/{My_Project}
  rv = !mkdir -p /content/{My_Project}
  gnss_dir = f'/content/{My_Project}/{My_Project}_GNSS/'
  pix_dir  = f'/content/{My_Project}/{My_Project}_pix/'
  sfm_dir  = f'/content/{My_Project}/{My_Project}_sfm/'
  prod_dir =  f'/content/{My_Project}/{My_Project}_products/'
  rv = !mkdir -p {gnss_dir}
  rv = !mkdir -p {pix_dir}
  rv = !mkdir -p {sfm_dir}
  rv = !mkdir -p {prod_dir}

  rv = !cd {gnss_dir}; mkdir -p aircraft aircraft/raw aircraft/rinex aircraft/exif aircraft/pix_pos aircraft/trajectories \
      bases  Ground_Control
  rv = !cd {pix_dir}; mkdir  -p raw jpg field_jpg50 
  rv = !cd {prod_dir}; mkdir -p Reports laz dsm ortho
  !tree {My_Project}
  print(My_Project," is setup.\nOperation Complete.")

In [ ]:
#@title P3.0 Zip a project directory for downloading. {form-width: "25%"}
#@markdown Paste or type a directory or file below to zip.
if __name__ == '__main__':
  Zip_Name = "" #@param {type:"string"}
  print(f'Zipping: {Zip_Name}  Standby.')
  !zip -qr {Zip_Name} {Zip_Name}/*
  print('Operation Completed.')


In [ ]:
#@title P2.0 (Optional) Unzip a Project {form-width: "25%"}
if __name__ == '__main__':
  Project_Zip_File = "" #@param {type:"string"}
  print(f'UnZipping {Project_Zip_File} Standby.')
  rv = ! cd /; unzip  -qo {Project_Zip_File} 
  print(f'Operation Completed.')
  if rv != []:
    print(rv[0])

In [ ]:
#@title P4.0 Display a Project Tree
if __name__ == '__main__':
  My_Project = "/content/Gdrive/Missions/Uas/2020-1008-Fenwick" #@param {type:"string"}
  Tree_ops = "Dirs Only" #@param ["Dirs Only", "All Files"]
  if Tree_ops == 'Dirs Only':
    ops ='-d'
  else:
    ops = ''
  
  !tree {ops} {My_Project}

# RINEX File Utilities.




## Convert CWW-PPK raw data to RINEX

In [ ]:
#@title A.1.1: Convert a CWW-PPK raw GPS data file to RINEX for processing. {form-width: "25%"}

#@markdown The first thing that must be done is converting your raw GNSS receiver data to RINEX.
#@markdown Use this cell to convert CWW-PPK raw data files to RINEX.  You need this for
#@markdown the UAS PPK data.  If you are using a second CWW-PPK in "Base Station" mode, you can
#@markdown it for that also.  If you are using another survey grade receiver, you will probably
#@markdown need to use tools specific to that receiver to convert it to RINEX.  RINEX is the required
#@markdown data format for use with this software package.
if __name__ == '__main__':
  Raw_File_Name = "/content/Gdrive/Missions/Uas/2020-1008-Fenwick/2020-1008-Fenwick_GNSS/bases/local-fenwick/2020-1008-173123-Fenwichbase-GP173104.RAW" #@param {type:"string"}
  Aircraft_RINEX_Dir = "/content/tmp" #@param {type:"string"}
  Data_Source   = "zebra" #@param ["Solo", "M600", "N7251F", "CCX", "TFR"] {allow-input: true}
  Marker_Name   = "zebra" #@param {type:"string"}
  User_Comment  = "2020-0927 Zebra" #@param {type:"string"}

  try:
    rtk_lib_loaded
    import re
    import os as os
    import georinex as gr
  except NameError:
    print('Run the above "**Step 0. **" cell above to configure tools.')
  else:
    if os.path.exists(Raw_File_Name):
      # Build up command string for the RTKlib convbin program.
      c1='-hc \''+User_Comment+'\' '
      c2='-hc \''+Data_Source+'\' '
      hm='-hm \''+Marker_Name+'\' '
      fn_root = str.split(Raw_File_Name,"/")[-1]
      fn_root = str.split(fn_root,'.')[0]
      cmd = '/usr/local/bin/convbin -os -od -f 2 -ti 1 -r nov '+c1+hm+Raw_File_Name
      print(cmd)
      print('Converting raw to RINEX')
      os.system( cmd )
      #!head -20 /content/GP212112.obs


      # Extract the base file name after all '/' and before any '.'
      orig_root_fn = os.path.basename(Raw_File_Name)
      orig_root_fn = os.path.splitext(orig_root_fn)[0]
      orig_path  = os.path.dirname(Raw_File_Name)

      # build up path name for the RINEX .obs file that should now exist
      fobs = orig_path+'/'+orig_root_fn+'.obs'

      # extract the RINEXT Header information to get the data & time and other fields
      print('Reading the RINEX obs, extracting data & time.')
      rnx=gr.rinexheader(fobs)
      rnx['t0'].date(), rnx['t0'].time()
      ofn = format(rnx['t0'], '%Y-%m%d-%H%M%S-')+Data_Source+'-'+fn_root

      # Rename all of the generate RINEX files using the computed root name.
      d = os.listdir(orig_path)
      print('Renaming all of the RINEX files')
      for i in d:
        if re.match( os.path.splitext(i)[0], orig_root_fn  ):
          ext = os.path.splitext(i)[1]
          new_fn = orig_path+'/'+ofn+ext
          new_fn = re.sub(r"[\n\t\s]*", "", new_fn)
          old_fn = orig_path+'/'+i
          print(old_fn, new_fn)
          os.rename(old_fn, new_fn)
      if Aircraft_RINEX_Dir !='':
        !mv {orig_path}/*obs {orig_path}/*.nav {orig_path}/*.gnav {Aircraft_RINEX_Dir}
      print('Operation completed.')
    else:
      print(Raw_File_Name+' not found.')

## Show Start, Stop and stats for a RINEX file.

In [ ]:
#@title Show Start, Stop times, stats of a RINEX obs file(s).
#@markdown Uses Teqc +qc to determine the start & stop time, and some stats.
if __name__ == '__main__':
  Path_Names = "/content/Gdrive/Missions/Uas/2020-1006-MARKS/2020-1006-MARKS_GNSS/aircraft/rinex/2020-1006-173431-White-GP173410.obs /content/Gdrive/Missions/Uas/2020-1006-MARKS/2020-1006-MARKS_GNSS/aircraft/rinex/2020-1006-173531-yellow-GP173510.obs /content/Gdrive/Missions/Uas/2020-1006-MARKS/2020-1006-MARKS_GNSS/aircraft/rinex/2020-1006-173700-Green-GP173640.obs  /content/Gdrive/Missions/Uas/2020-1006-MARKS/2020-1006-MARKS_GNSS/aircraft/rinex/2020-1006-191127-Zebra-all.obs" #@param {type:"string"}
  Path_Names = Path_Names.split()
  for fn in Path_Names:
    print(fn)
    junk = ! teqc +qc {fn} 2>/dev/null
    id  = junk.grep('4-character ID')
    beg = junk.grep('Time of ')[0]
    end = junk.grep('Time of ')[1]
    sumhead = junk.grep('first epoch')[0]
    summary = junk.grep('SUM')[0]
    print(f'{id}\n{beg}\n{end}\n{sumhead}\n{summary}\n')



## Teqc Quality Report

In [ ]:
#@title A.1.2 QC a RINEX Observation (*.Obs or *.19o) file using Teqc {form-width: "25%"}
#@markdown See [Page 34 of the UNAVCO Teqc Tutorial](https://www.unavco.org/software/data-processing/teqc/doc/UNAVCO_Teqc_Tutorial.pdf) for help interpreting the output.
if __name__ == '__main__':
  OBS_File_QC = "/content/tmp/2020-1008-190733-Fenwichbase-all.obs" #@param {type:"string"}
  !teqc +qc {OBS_File_QC} 2>/content/errors.txt

## Prepare RINEX data to send to OPUS

In [ ]:
#@title Prepare Base RINEX to send to OPUS or to use with RTKlib (static). { form-width: "25%" }
#@markdown If you setup a local base station using a 
#@markdown second CWW-PPK or other GNSS receiver you can
#@markdown use this cell to generate the necessary RINEX
#@markdown data files to send to 
#@markdown [OPUS](https://www.ngs.noaa.gov/OPUS/).
#@markdown [OPUS](https://www.ngs.noaa.gov/OPUS/) gives you
#@markdown the highest accuracy location
#@markdown of your base station.
#@markdown This cell can process raw CWW-PPK data or RINEX
#@markdown data from another GNSS receiver.
#@markdown It will  re-sample your base station data
#@markdown to a specified interval. A 30 second interval is 
#@markdown recommended.
#@markdown It is important to use the correct antenna type ID when
#@markdown when you process this.  Not doing so can cause
#@markdown the resulting positon to be in error by 10cm or more
#@markdown depending on your base station antenna make/model.
#@markdown Much more information on OPUS canbe found in this document: [OPUS_2012_MALSCE.pdf](https://outside.vermont.gov/agency/vtrans/external/docs/geodetic/OPUS_2012_MALSCE.pdf)


if __name__ == '__main__':
  Base_Marker_Name = "2020-0927-MARKS" #@param ["CWW_PPK_BASE", ""] {allow-input: true}
  OPUS_Sample_Interval_Seconds =  30 #@param {type:"integer"}
  Base_Antenna_Type = "CNTAT330" #@param ["CNTAT330", ""] {allow-input: true}
  Base_Receiver_Data_Format = "RINEX" #@param ["Raw-CWW-PPK", "RINEX"] {allow-input: true}
  Base_Raw_File = "/content/tmp/2020-1008-190733-Fenwichbase-GP190714.obs" #@param {type:"string"}
  Base_User_Comment = "" #@param {type:"string"}

  Base_Rx_Types = {
      'NovAtel OEMV/4/6,OEMStar'    : 'nov',
      'NovAtel OEM3'                : 'oem3',
      'ublox LEA-4T/5T/6T/7T/M8T'   : 'ubx',
      'Swift Navigation SBP'        : 'sbp',
      'Hemisphere Eclipse/Crescent' : 'hemis',
      'SkyTraq S1315F'              : 'stq',
      'Javad'                       : 'javad',
      'Trimble RT17'                : 'rt17',
      'Septentrio SBF'              : 'sbf',
      'BINEX'                       : 'binex',
      'TERSUS'                      : 'tersus',
      'RINEX'                       : 'rinex'
      }

  Rx_fmt = { 
            'Raw-CWW-PPK'   : 'nov',
            'RINEX'         : 'rinex'
            }


  opus_dir = os.path.dirname( Base_Raw_File )+'/opus'
  print('Generating RINEX data for OPUS.')
  !/usr/local/bin/convbin \
    -r {Rx_fmt[Base_Receiver_Data_Format]} \
    -hm {Base_Marker_Name} \
    -ti {OPUS_Sample_Interval_Seconds} \
    -tt 0.02 \
    -ha {Base_Antenna_Type} \
    -c {Base_Marker_Name} \
    -d {os.path.dirname( Base_Raw_File )+'/'}opus \
    {Base_Raw_File} 2> errors.txt
  !grep -h 'TIME OF' {opus_dir+'/*'}
  !grep -h 'ANT #'   {opus_dir+'/*'}
  print(opus_dir)
  ! ls -la {opus_dir}
  print('Operation completed.  Download the generated observation data and send to OPUS.')

## Concatenate RINEX files with teqc

In [ ]:
#@title A1.1.2: Concatenate RINEX files using teqc
if __name__ == '__main__':
  File_1 = "/content/Gdrive/Missions/Uas/2020-1008-Fenwick/2020-1008-Fenwick_GNSS/bases/local-fenwick/2020-1008-173123-Fenwichbase-GP173104.obs" #@param {type:"string"}
  File_2 = "/content/Gdrive/Missions/Uas/2020-1008-Fenwick/2020-1008-Fenwick_GNSS/bases/local-fenwick/2020-1008-190733-Fenwichbase-GP190714.obs" #@param {type:"string"} 
  File_3 = "" #@param {type:"string"}
  File_4 = "" #@param {type:"string"} 
  File_5 = "" #@param {type:"string"}
  File_6 = "" #@param {type:"string"}
  Output_file = "/content/tmp/concated-rinex.obs" #@param {type:"string"}
  !teqc {File_1} {File_2} {File_3} {File_4} {File_5} {File_6} >{Output_file}

# General Purpose File Utilities

## Head & Tail ***File***

In [ ]:
#@title Display the head and tail of a File. {form-width: "25%"}
##show_file_head = "Base_RINEX" #@param ['Base_RINEX', 'Base_RINEX_Nav', 'Base_GPS_SP3', 'Base_RINEX_Gnav', 'Base_GLO_SP3' ]
if __name__ == '__main__':
  show_file_head = "/content/Gdrive/Missions/Uas/2020-1008-Fenwick/2020-1008-Fenwick_GNSS/aircraft/trajectories/2020-1008-White-all-Fenwick-UK15N-cmb-pos.txt" #@param {type:"string"}
  Head_Lines_to_display =  27#@param ["20", "30", "40", "50", "75", "100"] {type:"raw", allow-input: true}
  Tail_Lines_to_display =  3#@param ["20", "30", "40", "50", "75", "100"] {type:"raw", allow-input: true}
  print('==========================================[ Head ] =========================================')
  !head -{Head_Lines_to_display} {show_file_head}
  print('\n========================================[ Tail ] =========================================')
  !tail -{Tail_Lines_to_display} {show_file_head}

## Show size of a ***path(s)***

In [ ]:
#@title ! du -skh {Path_Name}  Show path size.
if __name__ == '__main__':
  Path_Name = "/content/Gdrive/Missions/NC_Coast/2020-0928-NC_GNSS/bases/CORS  /content/Gdrive/Missions/NC_Coast/2020-0928-NC_GNSS/bases/CORS-old /content/Gdrive/Missions/NC_Coast/2020-0928-NC_GNSS/aircraft/rinex" #@param {type:"string"}
  ! du -skh {Path_Name}

## Head ***file***

In [ ]:
#@title Display the head of a File. {form-width: "25%"}
##show_file_head = "Base_RINEX" #@param ['Base_RINEX', 'Base_RINEX_Nav', 'Base_GPS_SP3', 'Base_RINEX_Gnav', 'Base_GLO_SP3' ]
if __name__ == '__main__':
  show_file_head = "/content/tmp/opus/2020-0927-MARKS2820.20O" #@param {type:"string"}
  Lines_to_display = 50 #@param ["20", "30", "40", "50", "75", "100"] {type:"raw", allow-input: true}
  !head -{Lines_to_display} {show_file_head}

## Tail ***file***

In [ ]:
#@title Display the tail of a File. {form-width: "25%"}
##show_file_head = "Base_RINEX" #@param ['Base_RINEX', 'Base_RINEX_Nav', 'Base_GPS_SP3', 'Base_RINEX_Gnav', 'Base_GLO_SP3' ]
if __name__ == '__main__':
  show_file_head = "/content/tmp/concated-rinex.obs" #@param {type:"string"}
  Lines_to_display = 50 #@param ["20", "30", "40", "50", "75", "100"] {type:"raw", allow-input: true}
  !tail -{Lines_to_display} {show_file_head}

## mv ***source_path destination_path***

In [ ]:
#@title ! mv -r Source_Path Destination_path
if __name__ == '__main__':
  Source_Path      = "" #@param {type:"string"}
  Destination_Path = "" #@param {type:"string"}  
  Flag = "Verbose" #@param ["Verbose", "Quiet"]
  if Flag == 'Verbose':
    q = 'v'
  else:
    q = ''   
  ! mv -{q}f {Source_Path} {Destination_Path}

## gzip -rv ***path_name***

In [ ]:
#@title ! gzip -rv Path_Name
if __name__ == '__main__':
  Path_Name = "" #@param {type:"string"}
  Flag = "Verbose" #@param ["Verbose", "Quiet"]
  if Flag == 'Verbose':
    q = 'v'
  else:
    q = '' 
  initial_size = ! du -skh {Path_Name}
  ! gzip -r{q} {Path_Name}
  final_size = ! du -skh {Path_Name}
  print(f'Starting: {initial_size}\n  Ending: {final_size}')

## gunzip -rv ***path_name***

In [ ]:
#@title ! gunzip -rv {Path_Name}
if __name__ == '__main__':
  Path_Name = "" #@param {type:"string"}
  Flag = "Quiet" #@param ["Verbose", "Quiet"]
  if Flag == 'Verbose':
    q = 'v'
  else:
    q = '' 
  initial_size = ! du -skh {Path_Name}
  ! gunzip -r{q} {Path_Name}
  final_size = ! du -skh {Path_Name}
  print(f'Starting: {initial_size}\n  Ending: {final_size}')


## cat ***file1 file2 file3 file4 file5 file6 >output_file***

In [ ]:
#@title Concatenate files using Linux cat
if __name__ == '__main__':
  File_1 = "/content/Gdrive/Missions/Uas/2020-0927-MARKS/2020-0927-MARKS_GNSS/aircraft/raw/white/2020-0927-185520-White-GP185502.TXT" #@param {type:"string"}
  File_2 = "/content/Gdrive/Missions/Uas/2020-0927-MARKS/2020-0927-MARKS_GNSS/aircraft/raw/white/2020-0927-202423-White-GP202404.TXT" #@param {type:"string"} 
  File_3 = "/content/Gdrive/Missions/Uas/2020-0927-MARKS/2020-0927-MARKS_GNSS/aircraft/raw/white/2020-0927-214122-White-GP214102.TXT" #@param {type:"string"}
  File_4 = "" #@param {type:"string"} 
  File_5 = "" #@param {type:"string"}
  File_6 = "" #@param {type:"string"}
  Output_file = "/content/Gdrive/Missions/Uas/2020-0927-MARKS/2020-0927-MARKS_GNSS/aircraft/raw/white/2020-0927white-all.TXT" #@param {type:"string"}
  !cat {File_1} {File_2} {File_3} {File_4} {File_5} {File_6} >{Output_file}

## rm ***path***

In [ ]:
#@title ! rm -rvf {Path_Name}
if __name__ == '__main__':
  Path_Name = "" #@param {type:"string"}
  Flag = "Verbose" #@param ["Verbose", "Quiet"]
  Force = "No" #@param ["Yes", "No"]
  
  if Force == 'Yes':
    f = ''
  else:
    f = 'i'

  if Flag == 'Verbose':
    q = 'v'
  else:
    q = ''
  if Path_Name != '':   
    ! rm -r{q}{f} {Path_Name}
  else:
    print('No file(s) specified.  Nothing to do.')

# User command cells